## DEVARSHI CHAUHAN

**<font color='red'> Data Exploration on DONORS CHOOSE Dataset </font>**

Here,
- NOTE: Resourse data(price and quantity) is murged with project_data to form new_project_data.  

In [149]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

from collections import Counter

#loading csv files

project_data = pd.read_csv(r'donorschoose-application-screening\train\train.csv')
resource_data = pd.read_csv(r'donorschoose-application-screening\resources\resources.csv')

print('Project data: ',project_data.shape)
print('Resource data: ',resource_data.shape)

project_data.head()

Project data:  (182080, 16)
Resource data:  (1541272, 4)


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,18-11-2016 14:45,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,26-04-2017 15:57,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,01-01-2017 22:57,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,12-08-2016 15:42,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,06-08-2016 09:09,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [150]:
# Mearging Price and Quantity column with new_project_data

#https://datascience.stackexchange.com/questions/33053/how-do-i-compare-columns-in-different-data-frames
price_df = resource_data.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()
print(price_df.head())

new_project_data = pd.merge(project_data, price_df, on='id', how='left') # new_project_data
print(new_project_data.shape)
new_project_data.head()

        id    price  quantity
0  p000001   459.56         7
1  p000002   515.89        21
2  p000003   298.97         4
3  p000004  1113.69        98
4  p000005   485.99         8
(182080, 18)


,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,18-11-2016 14:45,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1,299.98,6
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,26-04-2017 15:57,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,20.00,20
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,01-01-2017 22:57,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1,469.99,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,12-08-2016 15:42,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0,684.47,5
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,06-08-2016 09:09,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1,355.50,2


In [151]:
new_project_data.sort_values(by=['project_submitted_datetime'], inplace=True)
new_project_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity
35726,p014128,9a501a39da20f8d046ecc0ca7aa35f3c,Mrs.,NV,01-01-2017 00:10,Grades PreK-2,"Math & Science, Literacy & Language","Health & Life Science, Literacy",Books For Our Science Unit,My students come from backgrounds that are dif...,I am asking for books to help my students lear...,NaN,NaN,My students need books to read while we study ...,24,1,122.29,11
48187,p148328,d775b411b9739642cb156f69f67039b0,Mrs.,TX,01-01-2017 00:13,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Success for Reading and Mathematics,I teach at a Title One school. I have 18 wonde...,Students will be able to use the manipulatives...,NaN,NaN,My students need manipulatives for mathematics...,3,0,163.97,13
69618,p019595,2c6bcedcb20c0ebdb7374f75ba190c39,Ms.,CA,01-01-2017 00:15,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Supplies needed for classroom,My students have disabilities and are low inco...,These supplies will help with my students ever...,NaN,NaN,My students need supplies such as pencils and ...,5,0,129.37,13
21004,p182965,75dbc13f200d39ca93fb1ee6fe0b07e5,Mrs.,CA,01-01-2017 00:17,Grades 3-5,"Literacy & Language, Music & The Arts","Literacy, Visual Arts","\""Hats Off to Dr. Seuss\"" \r\nReading + Rhymin...",My students love to hear a good story! Many of...,I love introducing my students to memorable ch...,NaN,NaN,"My students need a variety of Dr. Seuss books,...",12,1,275.44,27
33048,p148212,2e1b073bfd61eabefba9c376236df02c,Ms.,WA,01-01-2017 00:32,Grades PreK-2,"Applied Learning, Music & The Arts","Early Development, Visual Arts",Get our Motors Running -- FINE MOTORS!,My students are an amazing group of 2nd grader...,"Many of my 2nd graders cannot hold scissors, c...",NaN,NaN,My students need supplies to create works of a...,17,0,299.97,29


In [154]:
# randomly selecting 50k points
reduced_dataset = new_project_data.sample(n=50000)
reduced_dataset.describe()

,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.165580,0.847700,296.086328,17.024220
std,27.940531,0.359315,367.937074,26.569976
min,0.000000,0.000000,0.900000,1.000000
25%,0.000000,1.000000,104.027500,4.000000
50%,2.000000,1.000000,203.430000,9.000000
75%,9.000000,1.000000,377.072500,21.000000
max,451.000000,1.000000,9999.000000,930.000000


## PROCESSING TEXT

In [155]:
# https://stackoverflow.com/a/47091490/4084039
import re


def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    phrase = phrase.replace('\\r', ' ')    
    phrase = phrase.replace('\\"', ' ')    
    phrase = phrase.replace('\\n', ' ')    
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase) 
    return phrase

In [156]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

### PREPROCESSING TITLE TEXT

In [157]:
sent = decontracted(reduced_dataset['project_title'].values[5]) 
# \r \n \t remove from string python: http://texthandler.com/info/remove-line-breaks-python/
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')

#remove spacial character: https://stackoverflow.com/a/5843547/4084039
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

### PREPROCESSING ESSAY TEXT

In [158]:
# merging all essays in one 'esssay' in train data
reduced_dataset["essay"] = reduced_dataset["project_essay_1"].map(str) + reduced_dataset["project_essay_2"].map(str) + reduced_dataset["project_essay_3"].map(str) + reduced_dataset["project_essay_4"].map(str)
df = reduced_dataset
df.drop(["project_essay_1", "project_essay_2", "project_essay_3", "project_essay_4"], axis=1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity,essay
54467,p160110,de5ad01826d30de65a9591e07250360d,Ms.,LA,17-09-2016 23:04,Grades 3-5,Literacy & Language,Literature & Writing,Printing Press Reading Groups,My students need bean bag chairs for small gro...,1,0,630.26,11,Imagine that you are a student and it is your ...
154884,p055323,8aacd567dbe685cda5e679ad71478800,Mrs.,CA,14-12-2016 19:26,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Cut the Clutter! Fire Up the Focus!,My students need an easel that holds chart pap...,6,1,149.00,1,My students come to school excited to learn ev...
148167,p046731,dd0053a87eff0920024f5fac3ca5ff59,Mrs.,MI,02-09-2016 13:41,Grades PreK-2,Literacy & Language,Literacy,Creative Classroom Organization,My students need C-Line Classroom Connector Sc...,8,1,148.04,8,I teach first grade in an inner city school. T...
55844,p065294,5b8b22396d505d54a6f8c471495e5f5f,Ms.,CO,20-07-2016 23:51,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literacy",Daily Learning Necessities!,"My students need composition notebooks, folder...",3,0,258.87,27,My kindergarten students are attending a Title...
164327,p096609,f2fa8b07f6bf8445ca6ae82951278d40,Ms.,NV,01-12-2016 15:09,Grades 9-12,Literacy & Language,Literature & Writing,Energized to Read & Write - \r\nReward System,My students need a variety of snacks as a posi...,1,1,270.42,10,"My students are motivated special education, E..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174612,p019092,ebbe5f7eab0a5f3a8f698ffef5e92908,Mr.,TX,10-10-2016 16:53,Grades 9-12,"Math & Science, History & Civics","Environmental Science, Social Sciences",We Wet Our Plants!,My students need more labs to tie in instructi...,1,0,1122.65,22,Our population is primarily latino students. ...
161762,p040352,b34f837cc9ad523425248a104c0192e7,Mr.,CA,24-08-2016 00:17,Grades PreK-2,Health & Sports,"Health & Wellness, Team Sports",PE Keeps our Brains and Bodies Healthy!,My students need PE and recess equipment to st...,4,1,207.71,8,A typical day in my classroom involves a lot o...
161988,p121366,358a01999f8f03bca678966329103127,Mrs.,CA,01-07-2016 20:11,Grades PreK-2,"Special Needs, Music & The Arts","Special Needs, Visual Arts",Paint Positively,My students need a creative space to tell thei...,0,1,329.47,14,"As a teacher in a high poverty area, my studen..."
76770,p189599,4b28bc08967e0b687de639a557fff4d5,Teacher,GA,28-04-2016 07:22,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Kindles for Kids!,My students need Kindles for learning! Our kin...,0,1,53.06,3,What we learn with pleasure we never forget.\r...


In [159]:
essay = decontracted(reduced_dataset['essay'].values[5])
essay = essay.replace('\\r', ' ') 
essay = essay.replace('\\"', ' ') 
essay = essay.replace('\\n', ' ')

print(essay)

I am an ELL teacher who shares a classroom with two other ELL teachers We work directly with 40 students who are receiving services along with other students who have progressed to the monitor stages or have exited the program In the afternoons I teach one newcomer English class and two small group reading intervention and social studies classes Our classroom is also used for smaller testing environments and one on one or small group language teaching Our goal is to create a relaxed and welcoming atmosphere that would encourage our students to excel in spite of their language barriers These students work very hard to make sense of the environment into which they are immersed Ranging from having lived in the United States for one month to their entire lives our students experience a wide array of challenges as they strive to reach proficiency in their second language Book clubs are a big part of our fifth and sixth grade curriculum Students have the opportunity to read the same text in 

In [160]:
# Combining all the above stundents 

from tqdm import tqdm
preprocessed_essays_train = []
# tqdm is for printing the status bar
for sentance in tqdm(reduced_dataset['essay'].values):
        sent = decontracted(sentance)
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_essays_train.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:22<00:00, 2176.41it/s]


## Splitting data

In [161]:
# train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reduced_dataset, reduced_dataset['project_is_approved'], test_size=0.30, stratify = reduced_dataset['project_is_approved'])

X_train.drop(['project_is_approved'], axis=1, inplace=True)
X_test.drop(['project_is_approved'], axis=1, inplace=True)

In [162]:
print("X_train: ",X_train.shape)
print("X_test: ",X_test.shape)
print('----------------------------')
print("y_train: ",y_train.shape)
print("y_test: ",y_test.shape)

X_train:  (35000, 18)
X_test:  (15000, 18)
----------------------------
y_train:  (35000,)
y_test:  (15000,)


# Cleaning catagorical data:

In [163]:
#cleaning subject_catagories
  
for X in [X_train, X_test]:
    catogories = list(X['project_subject_categories'].values)
    # remove special characters from list of strings python: https://stackoverflow.com/a/47301924/4084039

    # https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
    # https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
    # https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
    cat_list = []
    for i in catogories:
        temp = ""
        # consider we have text like this "Math & Science, Warmth, Care & Hunger"
        for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & Hunger"]
            if 'The' in j.split(): # this will split each of the catogory based on space "Math & Science"=> "Math","&", "Science"
                j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i.e removing 'The')
            j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math & Science"=>"Math&Science"
            temp+=j.strip()+" " #" abc ".strip() will return "abc", remove the trailing spaces
            temp = temp.replace('&','_') # we are replacing the & value into 
        cat_list.append(temp.strip())

    X['clean_subject_categories'] = cat_list
    X.drop(['project_subject_categories'], axis=1, inplace=True)

    from collections import Counter
    my_counter = Counter()
    for word in X['clean_subject_categories'].values:
        my_counter.update(word.split())

    cat_dict = dict(my_counter)
    sorted_cat_dict = dict(sorted(cat_dict.items(), key=lambda kv: kv[1]))


In [164]:
for Y in [X_train, X_test]:
    catogories = list(Y['project_subject_subcategories'].values)
    # remove special characters from list of strings python: https://stackoverflow.com/a/47301924/4084039

    # https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
    # https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
    # https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
    cat_list = []
    for i in catogories:
        temp = ""
        # consider we have text like this "Math & Science, Warmth, Care & Hunger"
        for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & Hunger"]
            if 'The' in j.split(): # this will split each of the catogory based on space "Math & Science"=> "Math","&", "Science"
                j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i.e removing 'The')
            j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math & Science"=>"Math&Science"
            temp+=j.strip()+" " #" abc ".strip() will return "abc", remove the trailing spaces
            temp = temp.replace('&','_') # we are replacing the & value into 
        cat_list.append(temp.strip())

    Y['clean_subject_subcategories'] = cat_list
    Y.drop(['project_subject_subcategories'], axis=1, inplace=True)

    from collections import Counter
    my_counter = Counter()
    for word in Y['clean_subject_subcategories'].values:
        my_counter.update(word.split())

    cat_dict = dict(my_counter)
    sorted_cat_dict2 = dict(sorted(cat_dict.items(), key=lambda kv: kv[1]))


## One Hot encoding of categorical data:

### 1. teacher prefix

### train

In [165]:
df = pd.DataFrame(data = X_train)
df_prefix_out= df[['teacher_prefix']]
df_prefix_out.head()

df_onehot_prefix_train = pd.get_dummies(df_prefix_out,prefix=['prefix'], drop_first=False)
print('X_train',df_onehot_prefix_train.head(5))

X_train         prefix_Dr.  prefix_Mr.  prefix_Mrs.  prefix_Ms.  prefix_Teacher
58915            0           0            1           0               0
41262            0           0            1           0               0
173526           0           0            0           1               0
25369            0           0            0           1               0
124646           0           0            1           0               0


### TEST

In [166]:
df = pd.DataFrame(data = X_test)
df_prefix_out= df[['teacher_prefix']]
df_prefix_out.head()

df_onehot_prefix_test = pd.get_dummies(df_prefix_out,prefix=['prefix'], drop_first=False)
print('X_test',df_onehot_prefix_test.head(5))

X_test         prefix_Dr.  prefix_Mr.  prefix_Mrs.  prefix_Ms.  prefix_Teacher
98830            0           1            0           0               0
109761           0           0            0           1               0
12244            0           0            0           1               0
150683           0           0            0           1               0
62920            0           0            0           0               1


### 2. School State

### train

In [167]:
df = pd.DataFrame(data = X_train)
df_state_out= df[['school_state']]
df_state_out.head()

df_onehot_state_train = pd.get_dummies(df_state_out,prefix=['state'], drop_first=False)

print('X_train',df_onehot_state_train.head(5))

X_train         state_AK  state_AL  state_AR  state_AZ  state_CA  state_CO  state_CT  \
58915          0         0         0         0         0         0         0   
41262          0         0         0         0         0         0         0   
173526         0         0         0         0         0         0         0   
25369          0         0         0         0         0         0         0   
124646         0         0         0         0         0         0         0   

        state_DC  state_DE  state_FL  ...  state_SD  state_TN  state_TX  \
58915          0         0         0  ...         0         0         0   
41262          0         0         1  ...         0         0         0   
173526         0         0         0  ...         0         0         0   
25369          0         0         0  ...         0         0         0   
124646         0         0         0  ...         0         0         0   

        state_UT  state_VA  state_VT  state_WA  state_WI  st

### test

In [168]:
df = pd.DataFrame(data = X_test)
df_state_out= df[['school_state']]
df_state_out.head()

df_onehot_state_test = pd.get_dummies(df_state_out,prefix=['state'], drop_first=False)

print('X_test',df_onehot_state_test.head(5))

X_test         state_AK  state_AL  state_AR  state_AZ  state_CA  state_CO  state_CT  \
98830          0         0         0         0         0         0         0   
109761         0         0         0         0         0         0         0   
12244          0         0         0         0         1         0         0   
150683         0         0         0         0         0         0         0   
62920          0         0         0         0         0         0         0   

        state_DC  state_DE  state_FL  ...  state_SD  state_TN  state_TX  \
98830          0         0         0  ...         0         0         0   
109761         0         0         0  ...         0         0         0   
12244          0         0         0  ...         0         0         0   
150683         0         0         0  ...         0         0         0   
62920          0         0         0  ...         0         0         0   

        state_UT  state_VA  state_VT  state_WA  state_WI  sta

### 3. Project Grade Catagory

### train

In [169]:
df = pd.DataFrame(data = X_train)
df_grade_out= df[['project_grade_category']]
df_grade_out.head()

df_onehot_grade_train = pd.get_dummies(df_grade_out,prefix=['grade'], drop_first=False)

print('X_train',df_onehot_grade_train.head(5))

X_train         grade_Grades 3-5  grade_Grades 6-8  grade_Grades 9-12  \
58915                  0                 0                  0   
41262                  0                 0                  0   
173526                 1                 0                  0   
25369                  0                 1                  0   
124646                 1                 0                  0   

        grade_Grades PreK-2  
58915                     1  
41262                     1  
173526                    0  
25369                     0  
124646                    0  


### test

In [170]:
df = pd.DataFrame(data = X_test)
df_grade_out= df[['project_grade_category']]
df_grade_out.head()

df_onehot_grade_test = pd.get_dummies(df_grade_out,prefix=['grade'], drop_first=False)

print('X_test',df_onehot_grade_test.head(5))

X_test         grade_Grades 3-5  grade_Grades 6-8  grade_Grades 9-12  \
98830                  0                 0                  0   
109761                 0                 0                  0   
12244                  1                 0                  0   
150683                 0                 0                  0   
62920                  0                 1                  0   

        grade_Grades PreK-2  
98830                     1  
109761                    1  
12244                     0  
150683                    1  
62920                     0  


### 4. Clean Subject Catagories

### train

In [171]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer(vocabulary=list(sorted_cat_dict.keys()), lowercase=False, binary=True)
categories_one_hot_train = vectorizer.fit_transform(X_train['clean_subject_categories'].values)
print(vectorizer.get_feature_names())

print('X_train-->',categories_one_hot_train.shape)

['Warmth', 'Care_Hunger', 'History_Civics', 'Music_Arts', 'AppliedLearning', 'SpecialNeeds', 'Health_Sports', 'Math_Science', 'Literacy_Language']
X_train--> (35000, 9)


### test

In [172]:
vectorizer = CountVectorizer(vocabulary=list(sorted_cat_dict.keys()), lowercase=False, binary=True)
categories_one_hot_test = vectorizer.fit_transform(X_test['clean_subject_categories'].values)
print(vectorizer.get_feature_names())

print('X_test-->',categories_one_hot_test.shape)

['Warmth', 'Care_Hunger', 'History_Civics', 'Music_Arts', 'AppliedLearning', 'SpecialNeeds', 'Health_Sports', 'Math_Science', 'Literacy_Language']
X_test--> (15000, 9)


### 5. Clean Subject Sub Catagories

### train

In [173]:
vectorizer = CountVectorizer(vocabulary=list(sorted_cat_dict2.keys()), lowercase=False, binary=True)
subcategories_one_hot_train = vectorizer.fit_transform(X_train['clean_subject_subcategories'].values)
print(vectorizer.get_feature_names())

print('X_train',subcategories_one_hot_train.shape)

['Economics', 'CommunityService', 'FinancialLiteracy', 'ParentInvolvement', 'ForeignLanguages', 'Extracurricular', 'Civics_Government', 'NutritionEducation', 'Warmth', 'Care_Hunger', 'SocialSciences', 'PerformingArts', 'CharacterEducation', 'TeamSports', 'Other', 'College_CareerPrep', 'Music', 'History_Geography', 'EarlyDevelopment', 'Gym_Fitness', 'ESL', 'Health_LifeScience', 'EnvironmentalScience', 'VisualArts', 'Health_Wellness', 'AppliedSciences', 'SpecialNeeds', 'Literature_Writing', 'Mathematics', 'Literacy']
X_train (35000, 30)


### test

In [174]:
vectorizer = CountVectorizer(vocabulary=list(sorted_cat_dict2.keys()), lowercase=False, binary=True)
subcategories_one_hot_test = vectorizer.fit_transform(X_test['clean_subject_subcategories'].values)
print(vectorizer.get_feature_names())

print('X_test',subcategories_one_hot_test.shape)

['Economics', 'CommunityService', 'FinancialLiteracy', 'ParentInvolvement', 'ForeignLanguages', 'Extracurricular', 'Civics_Government', 'NutritionEducation', 'Warmth', 'Care_Hunger', 'SocialSciences', 'PerformingArts', 'CharacterEducation', 'TeamSports', 'Other', 'College_CareerPrep', 'Music', 'History_Geography', 'EarlyDevelopment', 'Gym_Fitness', 'ESL', 'Health_LifeScience', 'EnvironmentalScience', 'VisualArts', 'Health_Wellness', 'AppliedSciences', 'SpecialNeeds', 'Literature_Writing', 'Mathematics', 'Literacy']
X_test (15000, 30)


# Numerical data:

### 1. Price

### train

In [175]:
from sklearn.preprocessing import StandardScaler

price_scalar = StandardScaler()
price_scalar.fit(X_train['price'].values.reshape(-1,1)) # finding the mean and standard deviation of this data
    

price_standardized = price_scalar.transform(X_train['price'].values.reshape(-1, 1))
df_price_standardized_train = pd.DataFrame(price_standardized, columns = ['price_standardized'],index=X_train.index)
X_train.insert(value=df_price_standardized_train, column = 'price_standardized', loc=18)
X_train.pop('price')

print('X_train-->',f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")
print(df_price_standardized_train.head())
df_price_standardized_train.shape

X_train--> Mean : 295.82321371428577, Standard deviation : 378.22156145710306
        price_standardized
58915            -0.561399
41262            -0.443849
173526           -0.737222
25369            -0.165969
124646           -0.479172


(35000, 1)

### test 

In [176]:
price_scalar.fit(X_test['price'].values.reshape(-1,1)) # finding the mean and standard deviation of this data
    

price_standardized = price_scalar.transform(X_test['price'].values.reshape(-1, 1))
df_price_standardized_test = pd.DataFrame(price_standardized, columns = ['price_standardized'], index=X_test.index)
X_test.insert(value=df_price_standardized_test, column = 'price_standardized', loc=18)
X_test.pop('price')

print('X_test-->',f"Mean : {price_scalar.mean_[0]}, Standard deviation : {np.sqrt(price_scalar.var_[0])}")
print(df_price_standardized_test.head())

X_test--> Mean : 296.70026133333334, Standard deviation : 342.7279619525449
        price_standardized
98830            -0.255830
109761           -0.195345
12244            -0.543406
150683           -0.712519
62920             1.858295


# Text data

### 1. Title of TEXT

### train

In [177]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_title_train = []
    # tqdm is for printing the status bar
for title in tqdm(X_train['project_title'].values):
        
        # https://gist.github.com/sebleier/554280
        title = ' '.join(e for e in title.split() if e.lower() not in stopwords)
        preprocessed_title_train.append(title.lower().strip())
    

100%|████████████████████████████████████████████████████████████████████████| 35000/35000 [00:00<00:00, 107232.18it/s]


### test

In [178]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_title_test = []
    # tqdm is for printing the status bar
for title in tqdm(X_test['project_title'].values):
        title = ' '.join(e for e in title.split() if e.lower() not in stopwords)
        preprocessed_title_test.append(title.lower().strip())

100%|████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 107907.79it/s]


### 2. Essay of TEXT

### TRAIN

In [179]:
# Combining all the above stundents 

from tqdm import tqdm
preprocessed_essays_train = []
# tqdm is for printing the status bar
for sentance in tqdm(X_train['essay'].values):
        sent = decontracted(sentance)
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_essays_train.append(sent.lower().strip())


100%|██████████████████████████████████████████████████████████████████████████| 35000/35000 [00:15<00:00, 2221.07it/s]


In [180]:
preprocessed_essays_train[1000]

'students traditional honors level english language arts 9th 12th graders already recognize value reading many discovering wonders active reading life goal encourage read explore new literature read least 10 minutes class day students required book times end year many read dozen books could ever whole class school rapidly changing suburban environment nearly half students come socioeconomically disadvantaged homes number risen 20 5 years population english language learners increasing swift pace well school open enrollment no local boundaries students area choose come know expectations high provide safe environment grow learn stephen king called books uniquely portable magic strong desire see students embrace magic first step provide choice ability choose variety books fiction nonfiction books native languages classics school library serves large population students regularly end swapping borrowing library enough copies satisfy need tiny classroom library currently stocked almost entir

### TEST

In [181]:
# Combining all the above stundents 

from tqdm import tqdm
preprocessed_essays_test = []
# tqdm is for printing the status bar
for sentance in tqdm(X_test['essay'].values):
        sent = decontracted(sentance)
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_essays_test.append(sent.lower().strip())
        
preprocessed_essays_test[1000]

100%|██████████████████████████████████████████████████████████████████████████| 15000/15000 [00:06<00:00, 2180.31it/s]


'students love knowing know open book students enjoy reading love reading variety material thrive information get books especially nonfiction books class 21 second graders full energy full questions lots fun students go title 1 school coastal georgia 82 students free reduced lunch although students may lacking economic ways make inquisitive knowledge students love read devoured books school year challenged teacher find new interesting books genres capture attention requested variety materials requested large number nonfiction books help new reading program next school year reader workshop also requested manila paper colored pencils art supplies empower students art art supplies enrich students learning potential reading subjects empowering learning using art well donating project helping enrich classroom library nonfiction books different levels help students succeed new reader workshop program also helping students get art supplies empower learning art empowerment help students learn 

In [182]:
preprocessed_essays_test[1001]

'knowledge one thing no one take away us love look face find great fact book something love not put book drawn 28 amazing students choose read done work school title school heart mesa arizona means lot families work hard get child school let alone provide books home read see need writing classroom scores writing low not foundations good writer strong teacher writing feel like tools teach stronger writers missing tools want students strong writers not come class naturally knowing write start year teaching parts speech using books point natural reading use anchor charts teach find text work stages writing good sentence order write great paragraph write different types writing take baby steps writing amazing paragraph know teach writing help students learn baby steps lack materials class help teach chart tablet anchor charts lessons reminder class write lesson add sentence strips parts speech different colors close reading journals helps process writing take notes good writer write drafts

## SENTIMENT ANALYSIS

### TRAIN

In [183]:

from tqdm import tqdm
import nltk
nltk.download('vader_lexicon') 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

neg_train = []
pos_train = []
neu_train = []
compound_train = []
id_train = []

for sentence in list(preprocessed_essays_train):
    #print(sentence)
    ss = sid.polarity_scores(sentence)
    
    neg_train.append(ss['neg'])
    pos_train.append(ss['pos'])
    neu_train.append(ss['neu'])
    compound_train.append(ss['compound'])


for i in X_train['id'].values:
    id_train.append(i)

print(len(id_train))
print(len(neg_train))

# create dataframe of 4 features

data = {'id': list(id_train), 'negative': list(neg_train), 'positive': list(pos_train), 'neutral': list(neu_train), 'compound': list(compound_train)}
data_sentiment_train = pd.DataFrame (data = data, index=None, columns= None, dtype=None, copy=False)

data_sentiment_train.head()



n_new_project_data_train = pd.merge(X_train, data_sentiment_train, on = 'id', how='left') # new_project_data
print(n_new_project_data_train.shape)
n_new_project_data_train.head()

sentiment_train_ = n_new_project_data_train.loc[:, ['negative', 'positive','neutral','compound']]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DEVARSHI\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


35000
35000
(35000, 22)


### TEST

In [184]:
sid = SentimentIntensityAnalyzer()

neg_test = []
pos_test = []
neu_test = []
compound_test = []
id_test = []

for sentence in list(preprocessed_essays_test):
    #print(sentence)
    ss = sid.polarity_scores(sentence)
    
    neg_test.append(ss['neg'])
    pos_test.append(ss['pos'])
    neu_test.append(ss['neu'])
    compound_test.append(ss['compound'])


for i in X_test['id'].values:
    id_test.append(i)

print(len(id_test))
print(len(neg_test))

# create dataframe of 4 features

data = {'id': list(id_test), 'negative': list(neg_test), 'positive': list(pos_test), 'neutral': list(neu_test), 'compound': list(compound_test)}
data_sentiment_test = pd.DataFrame (data = data, index=None, columns= None, dtype=None, copy=False)

data_sentiment_test.head()

# Mearging Price and Quantity column with new_project_data


n_new_project_data_test = pd.merge(X_test, data_sentiment_test, on = 'id', how='left') # new_project_data
print(n_new_project_data_test.shape)
n_new_project_data_test.head()

sentiment_test_ = n_new_project_data_test.loc[:, ['negative', 'positive','neutral','compound']]

15000
15000
(15000, 22)


<h1><font color='red'>Observartion :</font></h1>
<li>Sentiment analysis is performed over train, test and cv data set and added to the dataset as new features.</li>
<li>4 new features named 'negative', 'positive','neutral' and 'compound' were created.</li>

# TF-IDF

### TITLE

### train

In [185]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10)
title_tfidf_train = vectorizer.fit_transform(preprocessed_title_train)
print("Shape of X_train tfidf ",title_tfidf_train.shape)

Shape of X_train tfidf  (35000, 1628)


### test

In [186]:
title_tfidf_test = vectorizer.transform(preprocessed_title_test)
print("Shape of X_test tfidf ",title_tfidf_test.shape)

Shape of X_test tfidf  (15000, 1628)


### ESSAY

### train

In [187]:
vectorizer = TfidfVectorizer(min_df=10)
essay_tfidf_train = vectorizer.fit_transform(preprocessed_essays_train)
print("Shape of X_train tfidf ",essay_tfidf_train.shape)

Shape of X_train tfidf  (35000, 10485)


### test

In [188]:
essay_tfidf_test = vectorizer.transform(preprocessed_essays_test)
print("Shape of X_test tfidf ",essay_tfidf_test.shape)

Shape of X_test tfidf  (15000, 10485)


## AVG W2V

In [189]:
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
model = loadGloveModel(r'C:\Users\DEVARSHI\Desktop\glove.42B.300d.txt')

Loading Glove Model


1917494it [03:13, 9916.10it/s] 


Done. 1917494  words loaded!


In [190]:
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

### TITLE

### train

In [191]:
avg_w2v_vectors_title_train = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_title_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_title_train.append(vector)

print(len(avg_w2v_vectors_title_train))
print(len(avg_w2v_vectors_title_train[0]))

100%|████████████████████████████████████████████████████████████████████████| 35000/35000 [00:00<00:00, 110483.08it/s]


35000
300


### test

In [192]:
avg_w2v_vectors_title_test = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_title_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_title_test.append(vector)

print(len(avg_w2v_vectors_title_test))
print(len(avg_w2v_vectors_title_test[0]))

100%|████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 112493.87it/s]


15000
300


### ESSAY

### train

In [193]:
avg_w2v_vectors_essay_train = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_essay_train.append(vector)

print(len(avg_w2v_vectors_essay_train))
print(len(avg_w2v_vectors_essay_train[0]))

100%|██████████████████████████████████████████████████████████████████████████| 35000/35000 [00:06<00:00, 5214.04it/s]


35000
300


### test

In [194]:
avg_w2v_vectors_essay_test = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors_essay_test.append(vector)

print(len(avg_w2v_vectors_essay_test))
print(len(avg_w2v_vectors_essay_test[0]))

100%|██████████████████████████████████████████████████████████████████████████| 15000/15000 [00:02<00:00, 5147.90it/s]


15000
300


# TF-IDF weighted avg W2VEC

### TITLE

### train

In [195]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(preprocessed_title_train)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())


tfidf_w2v_vectors_title_train = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_title_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_title_train.append(vector)

print(len(tfidf_w2v_vectors_title_train))
print(len(tfidf_w2v_vectors_title_train[0]))

100%|█████████████████████████████████████████████████████████████████████████| 35000/35000 [00:00<00:00, 54535.57it/s]


35000
300


### test

In [196]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(preprocessed_title_test)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())


tfidf_w2v_vectors_title_test = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_title_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors_title_test.append(vector)

print(len(tfidf_w2v_vectors_title_test))
print(len(tfidf_w2v_vectors_title_test[0]))

100%|█████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 57922.40it/s]


15000
300


### ESSAY

### train

In [197]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(preprocessed_essays_train)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

tfidf_w2v_vectors2_essay_train = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors2_essay_train.append(vector)

print(len(tfidf_w2v_vectors2_essay_train))
print(len(tfidf_w2v_vectors2_essay_train[0]))

100%|███████████████████████████████████████████████████████████████████████████| 35000/35000 [00:52<00:00, 660.91it/s]


35000
300


### test

In [198]:
fidf_model = TfidfVectorizer()
tfidf_model.fit(preprocessed_essays_test)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

tfidf_w2v_vectors2_essay_test = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays_test): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    tf_idf_weight =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] # getting the vector for each word
            # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) # getting the tfidf value for each word
            vector += (vec * tf_idf) # calculating tfidf weighted w2v
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors2_essay_test.append(vector)

print(len(tfidf_w2v_vectors2_essay_test))
print(len(tfidf_w2v_vectors2_essay_test[0]))

100%|███████████████████████████████████████████████████████████████████████████| 15000/15000 [00:22<00:00, 680.02it/s]


15000
300


In [199]:
# removing project_essay_1/2/3/4 features from main dataset, and keeping only one combined essay feature.

n_new_project_data_train.pop('project_essay_1')
n_new_project_data_train.pop('project_essay_2')
n_new_project_data_train.pop('project_essay_3')
n_new_project_data_train.pop('project_essay_4')

n_new_project_data_test.pop('project_essay_1')
n_new_project_data_test.pop('project_essay_2')
n_new_project_data_test.pop('project_essay_3')
n_new_project_data_test.pop('project_essay_4')

n_new_project_data_train.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_title,project_resource_summary,teacher_number_of_previously_posted_projects,quantity,essay,clean_subject_categories,clean_subject_subcategories,price_standardized,negative,positive,neutral,compound
0,p033403,02c6e4d311ec2902505b925c60da168e,Mrs.,OK,26-11-2016 21:58,Grades PreK-2,Go BEYOND Writing: Let's PRINT IT!,My students need printer ink cartridges to hel...,33,11,"I have 40 wonderful smiling, friendly faces gr...",AppliedLearning Literacy_Language,EarlyDevelopment Literature_Writing,-0.561399,0.062,0.334,0.604,0.9929
1,p242112,c8b5f3c9ff076263e710a1a66bda1e00,Mrs.,FL,10-04-2017 15:13,Grades PreK-2,Let's Get Reading,My students need books and a special wiggle ch...,2,16,"My students love to learn, we are always readi...",Literacy_Language,Literature_Writing,-0.443849,0.016,0.407,0.577,0.9944
2,p138771,fa39d803656474a3d95a531c4493e88d,Ms.,ME,14-02-2017 11:55,Grades 3-5,Hear it!! Interact with it!! Learn it!!!,My students need comfortable headphones with n...,0,25,My classroom is a multi-grade leveled support ...,Literacy_Language,Literature_Writing,-0.737222,0.071,0.329,0.600,0.9927
3,p258622,1bdbed20521d6a8cf6a99f761d797a5d,Ms.,OR,31-05-2016 18:18,Grades 6-8,Chromebooks for our library!,My students need computers! As we shift towar...,0,8,Students at Hosford are a diverse bunch of stu...,Literacy_Language AppliedLearning,Literacy Other,-0.165969,0.055,0.066,0.879,0.4610
4,p122785,1f45d4820dd3424dedd474556af5a1b1,Mrs.,MN,05-10-2016 20:06,Grades 3-5,Pint-Sized Picassos Seek Art Supplies!,My students need these supplies to make amazin...,1,56,"My students are thinkers, dreamers, readers, c...",Music_Arts,VisualArts,-0.479172,0.033,0.338,0.630,0.9913


<h1><font color='red'>Observartion:</font></h1>
<li>Data is divided in train, test and cv before processing the data, and the cleaning process are stated neately (above) for all of them.</li>
<li>Text data is processed by glove model.</li>
<li>After processing essay 1/2/3/4 and combining them original essays were deleted from dataframe.</li>